In [28]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
import torch.optim as optim
import torch.nn as nn
import cv2  # OpenCV 사용
import numpy as np
from yolov5 import YOLOv5  # YOLOv5 라이브러리 사용 가정


ModuleNotFoundError: No module named 'yolov5'

In [ ]:
# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Using CPU")

# YOLOv5 모델 불러오기 (YOLOv5 모델 경로 필요)
# YOLOv5는 사전학습된 모델을 제공하며, 여기서는 "yolov5s.pt" 모델을 사용합니다.
# 이 모델은 COCO 데이터셋으로 사전학습되어 사람, 차량 등 여러 객체를 감지할 수 있습니다.
yolov5_model_path = "yolov5s.pt"  # YOLOv5 사전학습 모델 경로
yolo = YOLOv5(yolov5_model_path, device=device)


In [ ]:
# Custom Dataset Class 정의
class GenderStyleDataset(Dataset):
    def __init__(self, image_folder, label_mapping, transform=None):
        self.image_folder = image_folder
        self.image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg')]
        self.transform = transform
        self.label_mapping = label_mapping

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_folder, img_name)
        image = Image.open(img_path)

        # 파일명에서 성별과 스타일 추출
        parts = img_name.split('_')
        style = parts[3]  # 예: sportivecasual
        gender_code = parts[-1][0]  # M or W

        if gender_code == 'M':
            gender = 'man'
        elif gender_code == 'W':
            gender = 'woman'

        # 성별 & 스타일을 결합한 클래스 라벨
        class_label = f"{gender}_{style}"

        # 이미지에서 옷 부분만 추출 (YOLO 사용)
        image_np = np.array(image)  # PIL 이미지를 NumPy 배열로 변환
        results = yolo.predict(image_np)  # YOLO를 사용해 객체 감지 수행

        # 옷만 추출 (person 클래스가 있는 바운딩 박스를 사용한다고 가정)
        clothes_images = []
        for result in results:
            if result['class'] == 'person':
                x_min, y_min, x_max, y_max = result['box']
                cropped_image = image_np[y_min:y_max, x_min:x_max]
                clothes_images.append(cropped_image)
        
        # 여러 옷 이미지가 추출되었을 경우 첫 번째만 사용 (필요시 변경 가능)
        if clothes_images:
            image_cropped = Image.fromarray(clothes_images[0])
        else:
            image_cropped = image  # 옷이 감지되지 않으면 원본 사용

        # Transform 적용
        if self.transform:
            image_cropped = self.transform(image_cropped)

        # 문자열 라벨을 숫자 라벨로 변환
        label = self.label_mapping[class_label]

        return image_cropped, label


In [ ]:

# 라벨 인덱스 매핑 생성
def create_label_mapping(image_folder):
    label_set = set()
    for filename in os.listdir(image_folder):
        parts = filename.split('_')
        style = parts[3]
        gender_code = parts[-1][0]
        if gender_code == 'M':
            gender = 'man'
        elif gender_code == 'W':
            gender = 'woman'
        class_label = f"{gender}_{style}"
        label_set.add(class_label)
    label_mapping = {label: idx for idx, label in enumerate(sorted(label_set))}
    return label_mapping


In [ ]:

# 데이터 경로 및 변환 설정
training_image_dir = '/home/gyuha_lee/DCC2024/dataset/training_image'
validation_image_dir = '/home/gyuha_lee/DCC2024/dataset/validation_image'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [ ]:

# 라벨 매핑 생성
label_mapping = create_label_mapping(train_image_folder)
num_classes = len(label_mapping)

# 데이터셋 및 DataLoader 생성
train_dataset = GenderStyleDataset(image_folder=train_image_folder, label_mapping=label_mapping, transform=transform)
val_dataset = GenderStyleDataset(image_folder=val_image_folder, label_mapping=label_mapping, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=0, pin_memory=True)


In [ ]:

# ResNet-18 모델 정의 및 GPU로 전송
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, num_classes)  # 마지막 레이어 수정
model = model.to(device)


In [ ]:

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:

# 학습 루프 및 검증 루프
num_epochs = 50  # 에폭 수 설정
for epoch in range(num_epochs):
    # 에폭 시작 정보 출력
    print(f"Epoch {epoch + 1}/{num_epochs} 시작")

    # 학습 단계
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)  # GPU로 전송

        # Forward pass
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # 에폭 학습 완료 후 손실 출력
    print(f"Epoch [{epoch + 1}/{num_epochs}] 학습 완료, Average Loss: {running_loss/len(train_loader):.4f}")

    # 검증 단계
    model.eval()  # 모델 평가 모드로 설정
    correct = 0
    total = 0
    val_loss = 0.0
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)  # GPU로 전송

            val_outputs = model(val_inputs)
            val_loss += criterion(val_outputs, val_labels).item()

            _, predicted = torch.max(val_outputs, 1)
            total += val_labels.size(0)
            correct += (predicted == val_labels).sum().item()

    val_accuracy = 100 * correct / total
    val_loss = val_loss / len(val_loader)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # 다시 학습 모드로 전환
    model.train()

# 사전학습 모델을 사용하지 않는 경우, 특정 클래스(예: 옷, 사람 등)를 감지하도록 YOLO 모델을 직접 학습시켜야 합니다.
# 이를 위해서는 해당 클래스에 대한 라벨이 포함된 데이터셋이 필요하며, 이를 사용해 YOLO 모델을 학습해야 합니다.
# 학습된 YOLO 모델은 객체 감지 성능이 사전학습 모델보다 떨어질 수 있지만, 원하는 특정 객체에 대해 더 높은 정확도를 보일 수 있습니다.
